<a href="https://colab.research.google.com/github/samvass/capstone-model/blob/bert-text/Text_Classification_Bert_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 - Load Data

In [ ]:
!pip install accelerate
!pip install evaluate
!pip install transformers

In [ ]:
from datasets import load_dataset

In [ ]:
RawData = load_dataset("csv", data_files = {"train": "Dataset Train.csv", "test": "Dataset Test.csv"})

## 2 - Tokenize Data

In [ ]:
from transformers import AutoTokenizer

In [ ]:
Tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
TokenizeType = "OnlyText"

In [ ]:
def TokenizeData (Data) :

  match TokenizeType:
    case "OnlyTitle":
      return Tokenizer(Data["title"], truncation=True)
    case "OnlyText":
      return Tokenizer(Data["text"], truncation=True)
    case "JoinThenTokenize":
      pass
    case "TokenizeThenJoin":
      pass

In [ ]:
TokenizedData = RawData.map(TokenizeData, batched = True)

## Train and Test

In [ ]:
import evaluate
import numpy as np

from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
DataCollator = DataCollatorWithPadding(tokenizer = Tokenizer)

In [ ]:
Accuracy = evaluate.load("accuracy")

In [ ]:
def ComputeMetrics (eval):

    Predictions, Labels = eval
    Predictions = np.argmax(Predictions, axis = 1)
    return Accuracy.compute(predictions = Predictions, references = Labels)

In [ ]:
IDToLabel = {0: "Human", 1: "AI"}
LabelToId = {"Human": 0, "AI": 1}

In [ ]:
Model = AutoModelForSequenceClassification.from_pretrained (
    "distilbert-base-uncased", num_labels = 2, id2label = IDToLabel, label2id = LabelToId
)

In [ ]:
Args = TrainingArguments (

    output_dir = "0",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = False,

)

T = Trainer (

    model = Model,
    args = Args,
    train_dataset = TokenizedData["train"],
    eval_dataset = TokenizedData["test"],
    tokenizer = Tokenizer,
    data_collator = DataCollator,
    compute_metrics = ComputeMetrics,

)

T.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.494491,0.702128
2,No log,0.354689,0.971631


TrainOutput(global_step=44, training_loss=0.5063397667624734, metrics={'train_runtime': 2775.822, 'train_samples_per_second': 0.252, 'train_steps_per_second': 0.016, 'total_flos': 92727179059200.0, 'train_loss': 0.5063397667624734, 'epoch': 2.0})